## Online Simulation-Based Digital Twin: Nibelungenbrücke

This notebook introduces an online simulation-based digital twin developed for the Nibelungenbrücke bridge. The orchestration system is designed to interact dynamically with the user by gathering key inputs; such as time, physical parameters, and spatial positions to perform real-time simulations of the bridge structure.

The simulations are powered by the FenicSXConcrete package and support both thermomechanical and structural deflection analyses. This framework enables physics-based modeling and continuous digital monitoring of the bridge's behavior.




Please run the code below once to install the packages!

In [1]:
#import sys
#!{sys.executable} -m pip install git+https://github.com/BAMresearch/FenicsXConcrete pint gmsh pytest jsonschema pandas pyproj tqdm pvlib

Following code snippet adjusts the current working directory and modifies the system path to ensure proper module resolution.

In [2]:
import os
import sys

original_cwd = os.getcwd()
root_dir = os.getcwd()
orchestrator_dir = os.path.join(root_dir, 'nibelungenbruecke', 'scripts', 'digital_twin_orchestrator')
os.chdir(orchestrator_dir)
sys.path.insert(0, root_dir)

from nibelungenbruecke.scripts.digital_twin_orchestrator.orchestrator import Orchestrator

### Definition of input parameters

This dictionary defines the configuration for running a digital twin simulation, including model selection, temporal settings, sensor definitions, and simulation options.

In [3]:
simulation_parameters = {
    'simulation_name': 'TestSimulation',
    'model': 'TransientThermal_1',
    'start_time': '2023-08-11T08:00:00Z',
    'end_time': '2023-09-11T08:01:00Z',
    'time_step': '10min',
    'parameter_update': {'rho': 2800, 'E': 310000000000},       ##TODO: !!
    'virtual_sensor_positions': [
        {'x': 0.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor1'},
        {'x': 1.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor2'},
        {'x': 1.78, 'y': 0.0, 'z': 26.91, 'name': 'Sensor3'},
        {'x': -1.83, 'y': 0.0, 'z': 0.0, 'name': 'Sensor4'}
        # Note: the real sensor positions are added automatically by the interface, so you don't need to specify them here.
    ],
    'full_field_results': False, # Set to True if you want full field results, the simulation will take longer and the results will be larger.
    'uncertainty_quantification': False, # Set to True if you want uncertainty quantification, the simulation will take longer and the results will be larger.
}

The Orchestrator class acts as the central controller for managing the entire digital twin simulation workflow. It handles configuration, setup, execution, and post-processing of simulations based on the provided input parameters.<br>
<br>
Orchestrator initialization with respect to the given parameters:

In [4]:
orchestrator = Orchestrator(simulation_parameters)

Provide your key to MKP's API to registers it with the Orchestrator instance for secure communication with the backend services.

In [5]:
key=input("\nEnter the code to connect API: ").strip()
orchestrator.set_api_key(key)

After verifying that the virtual sensor coordinates lie within the mesh domain, the simulation is executed using:

In [6]:
orchestrator.load(simulation_parameters)
results = orchestrator.run()

All virtual sensors are within the mesh domain.
Info    : Reading '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/models/mesh_3d 1.msh'...
Info    : 2443 entities
Info    : 2197 nodes
Info    : 12920 elements
Info    : Done reading '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/models/mesh_3d 1.msh'
Same model with the same parameters!!


### Results
The result at the chosen sensors are plotted below. The results can be downloaded from the path written below.

In [7]:
orchestrator.plot_virtual_sensor_data()

In [8]:
# Results is probably just a dictionary of values with the sensor names as keys and the values as lists of results at each time step. --> Ask!  Previous one?
# The method should just be going over the keys defined as virtual sensors and plotting the results at those positions --> Ask!  Previous one?
# Passing the parameters as a dictionary to the method is also acceptable if needed.    --> Not done!!
# The option "uncertainty_quantification" is used to determine if the results should be plotted with uncertainty quantification or not.

#dt.plot_results_at_virtual_sensors(results, uncertainty_quantification=simulation_parameters['uncertainty_quantification'])
# The results file should be saved in an accessible path at the root, with metadata about the simulation and the results.
#print("PATH TO RESULTS:", dt.get_results_path())

We can plot aswell a comparison between model response and real sensors when available:

In [9]:
# orchestrator.plot_results_at_real_sensors(results, uncertainty_quantification=simulation_parameters['uncertainty_quantification'])  # Plot the results, this can be done after the run or separately if needed.

Execute the digital twin simulation using the loaded model and parameters. This will simulate the bridge behavior based on the selected physics model (e.g., thermal or displacement) and input configuration.

## Additional results (only if run)

### Full-field response (3D)

The full-field simulation results are currently available as files and can be downloaded from the specified paths.<br>
Interactive visualization within the interface is not yet supported but will be introduced soon.

In [10]:
orchestrator.plot_full_field_response()

Path to full-field results:
TransientThermal_1 -> h5py_path: ../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/output/paraview/Nibelungenbrücke_thermal.h5
TransientThermal_1 -> xmdf_path: ../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/output/paraview/Nibelungenbrücke_thermal.xmdf


In [11]:
# The 3D model results are generated by activating the "paraview" parameter in the simulation parameters, which will save the results as xdmf and h5 files.
# The visualization can be done using pyvista, which is a wrapper around VTK that allows for easy 3D visualization in Python.
# The results can be visualized using the plot_full_field_response method, which will load the results file and plot the full field response.
# If implementing this as a method does not work, we can just use the pyvista library directly to load the results file and plot the full field response.
# if simulation_parameters['full_field_results']:
#    print("Full field results are enabled, plotting the full field response.")
#    dt.plot_full_field_response(results, uncertainty_quantification=simulation_parameters['uncertainty_quantification'])  # Plot the full field response, this should be done using pyvista and the saved results file.
#    print("PATH TO FULL FIELD RESULTS:", dt.get_full_field_results_path())

In [12]:
from copy import deepcopy

new_parameters = deepcopy(simulation_parameters)
new_parameters['parameter_update'] = {'rho': 2800, 'E': 290000000000}

orchestrator.load(new_parameters)
#result2 = orchestrator.run()
orchestrator.plot_virtual_sensor_data()

All virtual sensors are within the mesh domain.


In [13]:


new_parameters = deepcopy(orchestrator.simulation_parameters)
new_parameters['virtual_sensor_positions'] = [
    {'x': 1.78, 'y': 0.0, 'z': 26.91, 'name': 'Sensor1'},
    {'x': -1.83, 'y': 0.0, 'z': 0.0, 'name': 'Sensor2'}
]

orchestrator.load(new_parameters)

result3 = orchestrator.run()
print("Third run result:", result3)
orchestrator.plot_virtual_sensor_data()

All virtual sensors are within the mesh domain.
Same model with the same parameters!!
Third run result: None


In [14]:
new_parameters['parameter_update'] = {'rho': 3000, 'E': 290000000000}
result4 = orchestrator.run(new_parameters)
print("Fourth run result:", result4)
orchestrator.plot_virtual_sensor_data()


New model 'thermal_model' saved successfully.
Info    : Reading '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/models/mesh_3d 1.msh'...
Info    : 2443 entities
Info    : 2197 nodes
Info    : 12920 elements
Info    : Done reading '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/models/mesh_3d 1.msh'


  2%|▏         | 96/4281 [00:32<24:46,  2.82it/s]

In [ ]:


result5 = orchestrator.run(new_parameters)
print("Fifth run result:", result5)
orchestrator.plot_virtual_sensor_data()


In [ ]:
os.chdir(original_cwd)
print("Working directory restored to:", original_cwd)